# Hermes — Run a basic Hartree-Fock energy calculation

See [the tutorial](/Tutorials/hermes_hartree_fock_energy_calculation.ipynb).

In [ ]:
# |hide
!rm -r ~/qdx/convert/ || true
!mkdir -p ~/qdx/convert/
%cd ~/qdx/hermes_hartree_fock_quickstart/

In [ ]:
# Get a pdb to work with - we use the pdb-tools cli here but you can download directly from rcsb.org
!pdb_fetch 1brs | pdb_selchain -A | pdb_delhetatm > 1B39_A_nohet.pdb
!ls

In [ ]:
# |hide
import json
import os
import rush
from pathlib import Path

WORK_DIR = Path.home() / "qdx" / "hermes_hartree_fock_quickstart"

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

In [ ]:
import rush

# Get our client, for calling modules and using the rush API
# - edit in your access token and API URL or set the RUSH_TOKEN and RUSH_URL environment variables
client = await rush.build_provider_with_functions(
    batch_tags=["hermes"],
)

# |hide_line
client = await rush.build_provider_with_functions(
    batch_tags=["hermes"], restore_by_default=True
)

(prepared_protein_qdxf, prepared_protein_pdb) = await client.prepare_protein(
    "1B39_A_nohet.pdb"
)

# There may be multiple conformers, so select the first one
(first_conformer,) = await client.pick_conformer(prepared_protein_qdxf, 0)

# Fragment the protein so that we can run a quantum energy calculation
(fragmented_protein,) = await client.fragment_aa(first_conformer, 1, "All")

# Arguments
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

fragmented_protein_out = await fragmented_protein.get()
fragmented_protein_contents = json.load(
    open(await fragmented_protein.download(), "r")
)

# Quantum energy calculation
(hermes_energy,) = await client.hermes_energy(
    fragmented_protein,
    {},
    {
        "basis": "STO-3G",
        "aux_basis": "6-31G",
        "method": "RestrictedRIMP2",
    },  # configuration for a fast converging, low accuracy run
    {
        "scf": {
            "max_iter": 50,
            "max_diis_history_length": 12,
            "convergence_metric": "DIIS",
            "dynamic_screening_threshold_exp": 10,
            "ndiis": 8,
            "niter": 40,
            "scf_conv": 0.000001,
        },
        "frag": {
            "cutoffs": {"dimer": 22},
            "cutoff_type": "Centroid",
            "level": "Dimer",
            "reference_fragment": len(
                fragmented_protein_contents["topology"]["fragments"]
            )
            - 1,
        },
    },
    resources=HERMES_RESOURCES,
)

energy = json.load(open(await hermes_energy.download(), "r"))
# Let's look at the expanded Hartree Fock energy
energy["expanded_hf_energy"]